In [2]:
import os
import tarfile
import urllib
import shutil
import re
import nltk
import string
import json

print(os.listdir())
print(os.listdir('unzipped'))

['.gitignore', 'AssignmentWeek8.ipynb', 'json', 'ElasticSearch.ipynb', 'unzipped', 'Latex_parsing.py', 'data_zipped', 'Latex_parsing.ipynb', 'text.rtf', 'elasticsearch-1.7.1', 'README.md', 'interface.py', '.ipynb_checkpoints', '.git', 'clean_scraper.ipynb']
['hep-th-2003', 'hep-th-1996', 'hep-th-2001', 'hep-th-2002', 'hep-th-1993', 'hep-th-1997', 'hep-th-1992', 'hep-th-1998', 'hep-th-1999', 'hep-th-1995', 'hep-th-2000', 'hep-th-1994']


In [3]:


contents = os.listdir()
# Should list among others: data_zipped, elasticserch-1.7.1, unzipped
# data_zipped should contain all hep-th-YYYY.tar.gz ziles
# unzipped should either be empty or filled with directories per year


def get_dataset():
    base = 'http://www.cs.cornell.edu/projects/kddcup/download/hep-th-'
    filebase = 'hep-th-'
    datafiles = [str(n) for n in range(1992, 2004)]
    for df in datafiles:
        testfile = urllib.request.URLopener()
        testfile.retrieve(base + df + '.tar.gz', 'data_zipped/' + filebase + df +'.tar.gz')

    
if 'unzipped' not in contents:
    os.mkdir('unzipped')
if 'data_zipped' not in contents:
    os.mkdir('data_zipped')
    if not len(os.listdir('data_zipped')):
        get_dataset()
if 'json' not in contents:
    os.mkdir('json')



In [4]:


zipped_folder = os.path.join(os.path.curdir, 'data_zipped')


def extract_tar():
    unzipped_folder = os.path.join(os.path.curdir, 'unzipped')
    datafiles = [str(n) for n in range(1991, 2004)]
    for tf in os.listdir(zipped_folder):
        tfi = os.path.join(zipped_folder, tf) # tarfile instance
        if True:
            print(tfi)
            if (tfi.endswith("tar.gz")):
                tar = tarfile.open(tfi, "r:gz")
                tar.extractall()
                tar.close()
            elif (tfi.endswith("tar")):
                tar = tarfile.open(tfi, "r:")
                tar.extractall()
                tar.close()
        shutil.move(tfi[-11:-7], os.path.join(unzipped_folder,tf[:-7]))
        
if not os.listdir('unzipped'):
    extract_tar()



In [5]:
testfile = 'unzipped/hep-th-2003/0301005'
latex_lines = []

with open(os.path.join(os.path.curdir, testfile)) as f:
    #print(''.join(f.readlines()))
    latex_lines = f.readlines()

#print(latex_lines)
print('\n'.join([line[:-1] for line in latex_lines]))
    

% hep-th/0301005
% Title: Structure of supergravity theories
%   Authors: Antoine Van Proeyen
%   Comments: 30 pages; contribution to the Proceedings of the XI Fall Workshop on Geometry and Physics, Oviedo
%    (Spain), September 23-25 (2002), to be published in the series "Publications of the Royal Spanish Mathematical
%    Society"
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% Contribution of Antoine Van Proeyen for the
% Proceedings of the XI Fall Workshop
% on Geometry and Physics, Oviedo (Spain), September 23-25 (2002)
%
%  "Structure of supergravity theories"
%
% to be published in the series "Publicaciones de la Real Sociedad
% Espa{\~n}ola de Matematicas" (Publications of the Royal Spanish Mathematical
% Society)."
%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\documentclass[11pt,twoside]{article}
\usepackage{amssymb}
\usepackage{latexsym}
\usepackage{slashed,cite}
\usepackage{lscape}
\textheight 195mm \textwidth 130mm \oddsidemargin 14

In [24]:

def strip_comments(ll):
    """ returns lines of the latex file, except that comments, i.e. everything behind % on a
    
    line, are removed.
    """
    line_list = ll[:]
    for ind, line in enumerate(line_list):
        comment_pos = line.find('%')
        if comment_pos != -1:
            line_list[ind] = line[:line.find('%')]
        else:
            line_list[ind] = line[:-1]
    line_list = [line.strip() for line in line_list]
    line_list = [line for line in line_list if line]
    return line_list
            

    
def get_name_section(section_string, depth):
    """ returns (sub) section name, string
    
    :param section_string: string, latex_line with section tag on it
    :param depth: int, depth of section 0 = \section, 1 = \subsection etc.
    :return: string, either what is included in the section tag, or all of it that is included on that line
    
    # TODO:
      -  check for closing tag beforehand, such that complete names can be extracted, if the name spans multiple rows
      -  simplify function by replacing the string in find for start with levelname(depth) (works now but tired, so not risking)
    """
    start = section_string.find('\\' + 'sub'* (depth) + 'section{')
    end = section_string.find('}')
    truestart = start + 3*depth + 9 #adding the latex command to index
    return section_string[truestart:end]


def levelname(depth):
    """ returns string with appropriate (sub) section latex command for given depth
    
    :param depth: int
    """
    return '\\' + 'sub'*depth + 'section{'

def section_delimiters(linelist, level):
    """ returns two lists, delimiters for (sub) sections as indices of the list,
    
    and their accompanying names
    """
    indices = []
    names = []
    lvln = levelname(level)
    for i, line in enumerate(linelist):
        if line.find(lvln) != -1:
            indices.append(i)
            names.append(get_name_section(line, level))
    return indices + [-1], names




class Node:
    """ the basic unit that will be dictionaryfied, currently each node containstheir own name (section title), 
    
    a list of subnodes and accompanying names.
    in the future the headnode (in the parsetree) will function as the main node, containing 
    subnodes 'recursively', as well as being the location in which other keywords and values are placed
    (i.e. 'author':[authorlist], 'date':'date' (in multiple formats for search))

    each of these keyswords gives more options to the user later on, so if 
    you have some 'free' time... (ha wouldn't you wish)
    this is a TODO as well
    
    I emphasize: if this is not done well, our search is going to seriously suck.
    
    """
    def __init__(self, name, linelist, level, headnode=False):
        self.tot_lines = len(linelist)
        self.headnode = headnode
        self.name = name
        self.linelist = linelist
        leveldelimiters, childnames =  section_delimiters(linelist, level)
        self.ld, self.cnames = leveldelimiters, childnames
        self.cn = []
        #print(level, self.name)

        if headnode:
            self.other_keys = {}
            tempkeys = KEYTAGS[:]
            tempkeynames = [tk[0] for tk in tempkeys]
            for ind, line in enumerate(linelist):
                name, content = self.extract_tags(line, ind, tempkeys)
                if name:
                    if not len(name):
                        print("wowzers")
                    temp_n_ind = tempkeynames.index(name)
                    tempkeys.pop(temp_n_ind)
                    tempkeynames.pop(temp_n_ind)
                    self.other_keys[name] = content.translate(PUNCTUATION_TABLE)
            if 'content' in tempkeynames:
                everything = " ".join(linelist).translate(PUNCTUATION_TABLE)
                #print(everything[:100])
                self.other_keys['content'] = everything
                
            #for k,v in self.other_keys.items():
                #print(k, v, "\n")

            

    def extract_tags(self, line, ind, tempkeys):
        """
        
        KEYTAGS have format [["name", "latex_begin_tag", "latex_end_tag"], ...]
        """
        found_lbt = ""
        found_let = ""
        found_n = ""
        close = 0
        content = ""
        for i, keytag in enumerate(tempkeys):
            n, comb = keytag
            for lbt, let in comb:
                i = line.find(lbt)

                if i != -1:
                    #print(lbt, line)
                    try:
                        close = self.find_closing(let, ind)
                    except IndexError:
                        print(keytag, ind)
                        raise IndexError

                    found_lbt = lbt
                    found_let = let
                    found_n = n
                    #print(found_lbt)
                    content = " ".join(self.linelist[ind:close])
                    break
                
        return found_n, content
                
                
    def find_closing(self, closetag, startindex):
        if closetag == '}':
            brace_count = 0
            while startindex < self.tot_lines:
                #print("startindex: ", startindex)
                if self.linelist[startindex].find('{') != -1:
                    brace_count += 1
                if self.linelist[startindex].find('}') != -1:
                    brace_count -= 1
                    if brace_count == 0:
                        return startindex + 1
                    else:
                        startindex += 1
                else:
                    startindex += 1
        else:    
            while startindex < self.tot_lines:
                if self.linelist[startindex].find(closetag) != -1:
                    return startindex + 1
                else:
                    startindex += 1

            
         

class parsetree:
    """ treelike structure, contains nodes
    
    This parsetree will later be converted to JSON,
    For nor it contains lists with childnodes at each node, each of these childnodes is a lower level section
    This will be converted to the json format later (with perhaps an intermediary dictionary format)
    """
    def __init__(self, tex_dir, json_dir, subdir, documentID, overwrite=False, _print=False):
        """ 

        :param linelist: list of lines from a latex file, stripped from comments and everything before 
            the \makefile tag, as \newcommand shenanigans make life difficult otherwise.
        :param documentID: string, the filename
        """
        latex_lines = []
        tfname = os.path.join(os.path.join(tex_dir, subdir), documentID)
        jsubdir = os.path.join(json_dir, subdir)
        jfname = os.path.join(jsubdir, documentID)
        
        if os.path.exists(jfname):
            if not overwrite:
                print("already parsed, going to next")
                return 
        else:
            if not os.path.exists(jsubdir):
                os.mkdir(jsubdir)


        with open(tfname, 'r', encoding='utf-8') as f:
            latex_lines = f.readlines()

        #no_comments = strip_comments(latex_lines)
        #self.headnode = Node(documentID, no_comments, 0, headnode=True)
        self.headnode = Node(documentID, latex_lines, 0, headnode=True)
        JSON = JSONify(self.headnode)
        
    
        with open(jfname, 'w') as f:
            print(jfname)
            #print(str(json.dumps(JSON)))
            f.write(str(json.dumps(JSON)))
        
def JSON_unknown_cn(node):
    if type(node) == str:
        no_punct = node.translate(PUNCTUATION_TABLE) #remove punctuation
        return no_punct
    else:
        return {node.name : [JSON_unknown_cn(cn) for cn in node.cn]}

def JSONify(node):
    base = {node.name : JSON_unknown_cn(node)}
    if node.headnode:
        base.update(node.other_keys)
    return base

def JSONify_str(node):
    return node.cn

        

        
# KEYTAGS have format ["name", [["latex_begin_tag_option1", "latex_end_tag_option1"], ["latex_begin_tag_option1", "latex_end_tag_option1"]]]
KEYTAGS = [["date",[["\date{", "}"]]],
           ["abstract",[["\\begin{abstract}", "\end{abstract}"], ["\section{abstract}", "\section{}"]]], 
           ["keywords", [["{\\bf Key words:}", "."]]],
           ["author", [["\\author{", "}"]]],
           ["keywords",[["\\it Key words:","\end"]]],
           ["content", [["\\section{", "\end{document}"]]],
           ["introduction", [["\\newsec{introduction", "\\newsec"],   # includes first line of section after intro
                             ["\\section{introduction", "\\section{"] # includes first line of section after intro
                    ]
                ]
          ]




# KEYTAGS have format ["name", ["latex_begin_tag", "latex_end_tag"]]
KEYTAGS = [["date",[["date{", "}"], #9301009, 00010001
                    ["Date{", "}"], #9301008
                    ["%Date: ", "\n"]]], # 9201002, 9201003, 9201004 etc...
                                        # Kun je checken of 9301010 "Mon Jan  4 13:06:16 1993" geeft? Of empty/niks?,
           ["abstract",[["\\begin{abstract}", "\\end{abstract}"], ["\\abstract{", "}"], ["\\Abstract{", "}"],
                        ["Abstract", "\\new"], ["abstract", "\\new"], ["abstract", "\\end"], ["Abstract", "\\end"],
                        ["\abstract{", "}"],]],
           ["author", [["\\author{", "}"], # 9201005
                        ["\\author{", "\\"], # 9201007, 9201008
                        ["author{", "}"], #9301009
                        ["author {", "}"], #9301006
                        ["%From: ", "\n"]]], # 9201004, 9301004,
           ["keywords",[["\\it Key words:","\\end"], ["\\Key words:","\\end"], ["Key words", "."]]],
           ["content", [["\\section{", "\\end{document}"]]],
           ["introduction", [["\\newsec{", "\\newsec"],   # includes first line of section after intro
                             ["\\section{", "\\section{"] # includes first line of section after intro
                    ]
                ]
          ]

PUNCTUATION_TABLE = str.maketrans({key: " " for key in string.punctuation})


tex_dir = "unzipped"
docID = "0301005"
json_dir = "json"
sub_dir = "hep-th-2003"
doctree = parsetree(tex_dir, json_dir, sub_dir, docID, True)


json/hep-th-2003/0301005


In [25]:
json_dir = "json"
tex_dir = "unzipped"

faulty = []

for sd in os.listdir("unzipped"):
    if not (sd == "hep-th-1992"):
        if not (sd == "hep-th-1993"):
            continue
    for file in os.listdir(os.path.join(tex_dir, sd)):
        try:
            parsetree(tex_dir, json_dir, sd, file, overwrite=True)
        except UnicodeDecodeError:
            faulty.append(file)
print(faulty)
        

 Paper  hep th 9306138
  From  Fedor Malikov  malikov kusm kyoto u ac jp 
  Date  Sat  26 Jun 93 09 
json/hep-th-1993/9306138
 Paper  hep th 9309043
  From  dowker v2 ph man ac uk
  Date  Wed  8 Sep 1993 11 32 03  0100
  Date 
json/hep-th-1993/9309043
json/hep-th-1993/9308060
json/hep-th-1993/9303019
 Paper  hep th 9309134
  From  Martin Schlichenmaier  schlichenmaier math uni mannheim de 
  Date  F
json/hep-th-1993/9309134
json/hep-th-1993/9311042
json/hep-th-1993/9307131
 Paper  hep th 9310078
  From  yamada math metro u ac jp
  Date  Thu  14 Oct 93 18 33 15 JST
 
 
  m
json/hep-th-1993/9310078
 Paper  hep th 9308080
  From  dij s a amtp liv ac uk
  Date  Tue  17 Aug 1993 14 18 17  0000  BST 

json/hep-th-1993/9308080
json/hep-th-1993/9304090
json/hep-th-1993/9301041
json/hep-th-1993/9308091
 Paper  hep th 9303104
  From  Per Berglund  BERGLUND sbitp itp ucsb edu 
  Date  Thu  18 Mar 1993 1
json/hep-th-1993/9303104
json/hep-th-1993/9312030
json/hep-th-1993/9308043
 Paper  hep th 9305

 Paper  hep th 9307064
  From  ARATYN IFT UESP ANSP BR UICVM UIC EDU
  Date  Fri  9 Jul 1993 08 58 B
json/hep-th-1993/9307064
json/hep-th-1993/9312083
json/hep-th-1993/9308154
json/hep-th-1993/9303058
json/hep-th-1993/9307052
json/hep-th-1993/9308020
 Paper  hep th 9307051
  From  PARANJ LPS UMONTREAL CA
  Date  Wed  7 Jul 1993 15 50 31  0400  EDT 

json/hep-th-1993/9307051
 Paper  hep th 9311071
  From  89 21141 shaddam usb ve  Antonio Gabriel Oliveira Moreira 
  Date  Th
json/hep-th-1993/9311071
 Paper  hep th 9301022
  From  li het brown edu  Miao Li 
  Date  Fri  8 Jan 93 01 15 16 EST
  Date 
json/hep-th-1993/9301022
 Paper  hep th 9312007
  From  SASHA UILGT1 PHYSICS UIUC EDU
  Date  Wed  1 Dec 1993 15 59 25  0600 
json/hep-th-1993/9312007
json/hep-th-1993/9304031
 Paper  hep th 9303009
  From  bantay hal9000 elte hu  Peter Bantay 
  Date  Tue  2 Mar 93 15 41 41

json/hep-th-1993/9303009
 Paper  hep th 9310058
  From  fendley voltaire usc edu  Paul Fendley 
  Date  Mon  11 Oct 93 

 Paper  hep th 9301083
  From  vafa string harvard edu  Cumrun Vafa 
  Date  Tue  19 Jan 93 17 40 31
json/hep-th-1993/9301083
json/hep-th-1993/9306113
json/hep-th-1993/9311128
 Paper  hep th 9304115
  From  POPE PHYS TAMU EDU
  Date  Sat  24 Apr 1993 0 07 31 CDT
 
           
json/hep-th-1993/9304115
 Paper  hep th 9308061
  From  Bill Unruh  unruh physics ubc ca 
  Date  12 Aug 93 12 21  0700
 
 
 
json/hep-th-1993/9308061
json/hep-th-1993/9304029
json/hep-th-1993/9312100
json/hep-th-1993/9302112
 Paper  hep th 9302132
  From  Jisuke Kubo  jik hep s kanazawa u ac jp 
  Date  Fri  26 Feb 93 15 55
json/hep-th-1993/9302132
 Paper  hep th 9308099
  From  hamberh uciph0 ps uci edu  Herbert W  Hamber 
  Date  Fri  20 Aug 93 
json/hep-th-1993/9308099
 Paper  hep th 9309025
  From  mbor phyq1 physik uni freiburg de  Martin Bordemann  3768  
  Date  F
json/hep-th-1993/9309025
 Paper  hep th 9301099
  From  POPE PHYS TAMU EDU
  Date  Sun  24 Jan 1993 22 12 54 CST
  Date  revi
json/hep-th-1993/9

json/hep-th-1993/9302131
json/hep-th-1993/9312002
 Paper  hep th 9303069
  From  SCHIMMRI sbitp itp ucsb edu
  Date  Thu  11 Mar 1993 00 45 PST
 
 
  
json/hep-th-1993/9303069
json/hep-th-1993/9306001
 Paper  hep th 9309093
  From  dunbar physics ucla edu  Dunbar 
  Date  Thu  16 Sep 93 17 05 44 PDT

json/hep-th-1993/9309093
json/hep-th-1993/9305184
 Paper  hep th 9312095
  From  Michelle Bourdeau  bourdeau thphys ox ac uk 
  Date  Sun  12 Dec 93 2
json/hep-th-1993/9312095
json/hep-th-1993/9312087
 Paper  hep th 9301048
  From  mokhov trbilun BITNET pucc Princeton EDU  Oleg Mokhov 
  Date  Tue  1
json/hep-th-1993/9301048
 Paper  hep th 9303143
  From  samson guinness ias edu  Samson Shatashvili 
  Date  Thu  25 Mar 93 1
json/hep-th-1993/9303143
 Paper  hep th 9310153
  From  sadov string harvard edu  Vladimir Sadov 
  Date  Fri  22 Oct 93 19 3
json/hep-th-1993/9310153
 Paper  hep th 9303074
  From  RRK PHYS TAMU EDU
  Date  Thu  11 Mar 1993 20 54 34 CST
 
  input har
json/hep-th-1993/9

 Paper  hep th 9310159
  From  sfetsos ruuntl fys ruu nl  Konstadinos Sfetsos 
  Date  Sat  23 Oct 9
json/hep-th-1993/9310159
json/hep-th-1993/9304009
 Paper  hep th 9309082
  From  ereidell marie mit edu  Evan Reidell 
  Date  Tue  14 Sep 1993 17 03 
json/hep-th-1993/9309082
 Paper  hep th 9303094
  From  Ken ichiro Aoki  aoki madonna physics ucla edu 
  Date  Tue  16 Mar 9
json/hep-th-1993/9303094
json/hep-th-1993/9303015
json/hep-th-1993/9306146
 Paper  hep th 9306028
  From  John Cardy  cardy sarek physics ucsb edu 
  Date  Fri  4 Jun 93 09 24
json/hep-th-1993/9306028
 Paper  hep th 9303114
  From   Gil N  Gat   gat wam umd edu 
  Date  Fri  19 Mar 1993 12 31 22  050
json/hep-th-1993/9303114
 Paper  hep th 9310198
  From  denjoe stp dias ie
  Date  Sun  31 Oct 93 17 33 32 GMT
 
  Environmen
json/hep-th-1993/9310198
 Paper  hep th 9311007
  From   Tze Dan Chung   chung puhep1 Princeton EDU 
  Date  Mon  1 Nov 93 16
json/hep-th-1993/9311007
json/hep-th-1993/9302035
json/hep-th-1993/9

...

json/hep-th-1992/9212006
 Paper  hep th 9206099
  From  JALFARO CERNVM BITNET pucc princeton edu
  Date  Thu  25 Jun 92 16 10
json/hep-th-1992/9206099
 Paper  hep th 9212019
  From   David Lowe   lowe puhep1 Princeton EDU 
  Date  Wed  2 Dec 92 20 04 
json/hep-th-1992/9212019
 Paper  hep th 9211074
  From  Franco Ferrari LS WESS PHYSIK UNI MUENCHEN DBP DE
  Date  Mon  16 Nov
json/hep-th-1992/9211074
 Paper  hep th 9209073
  From  SALEUR YALPH2 bitnet yalevm ycc yale edu
  Date  21 Sep 1992 11 37 35
json/hep-th-1992/9209073
 Paper  9203055
  From  MARTELLINI MILANO INFN IT icineca cineca it
  Date  Fri  20 MAR 92 13 17 N
 
json/hep-th-1992/9203055
 Paper  hep th 9212073
  From  AMBJORN nbivax nbi dk
  Date  11 Dec 1992 15 33 49  0100
 
  
   STAN
json/hep-th-1992/9212073
 Paper  hep th 9210086
  From  TAKEBE Takashi  takebe math s u tokyo ac jp 
  Date  Fri  16 Oct 92 1
json/hep-th-1992/9210086
 Paper  9201045
  From  RRK phys tamu edu
  Date  Wed  22 Jan 1992 15 49 30 CST
 
  input har

json/hep-th-1992/9205063
 Paper  hep th 9207034
  From  Steve Giddings  giddings denali physics ucsb edu 
  Date  Fri  10 Jul
json/hep-th-1992/9207034
 Paper  hep th 9205057
  From  DADDA torino infn it
  Date  Mon  18 MAY 92 12 07 GMT
 
             
json/hep-th-1992/9205057
 Paper  hep th 9207030
  From  Thomas Larsson  tl theophys kth se 
  Date  Thu  9 Jul 92 17 05 32 ED
json/hep-th-1992/9207030
 Paper  hep th 9206016
  From  SEN tifrvax bitnet cunyvm cuny edu
  Date  Thu  4 Jun 92 00 09 IST  0
json/hep-th-1992/9206016
 Paper  hep th 9204092
  From  LDICKEY nsfuvax math uoknor edu
  Date  Tue  28 Apr 1992 14 17 CDT
 

json/hep-th-1992/9204092
json/hep-th-1992/9201056
json/hep-th-1992/9201030
 Paper  hep th 9210093
  From  NISHINO umdhep BITNET VTVM2 CC VT EDU
  Date  Fri  16 Oct 92 16 28 ED
json/hep-th-1992/9210093
 Paper  hep th 9208074
  From  BDEWIT CERNVM BITNET pucc princeton edu
  Date  Fri  28 Aug 92 12 05 
json/hep-th-1992/9208074
json/hep-th-1992/9205005
json/hep-th-1992/9

 Paper  9201038
  From  FEHER LPS UMONTREAL CA
  Date  Mon  20 Jan 1992 22 18 51 EST
  Date  revised
json/hep-th-1992/9201038
 Paper  hep th 9210123
  From  griffies cvetic hep upenn edu
  Date  Thu  22 Oct 1992 12 14 05  0400
json/hep-th-1992/9210123
 Paper  hep th 9212031
  From  xmolin PLUNLO51 BITNET vm cnuce cnr it
  Date  Fri  04 Dec 1992 10 45
json/hep-th-1992/9212031
 Paper  9202034
  From  klassen strange tn cornell edu  Tim Klassen 
  Date  Mon  10 Feb 92 23 12 22
json/hep-th-1992/9202034
 Paper  hep th 9209052
  From  VENEZIA crnvma cern ch
  Date  Tue  15 Sep 92 16 54 51 SET
 
  magnif
json/hep-th-1992/9209052
json/hep-th-1992/9207013
json/hep-th-1992/9207084
 Paper  9203054
  From  LARUS slacvm slac stanford edu
  Date  Thu  19 Mar 1992 17 55  0800  PST 
 

json/hep-th-1992/9203054
 Paper  hep th 9207020
  From  AMBJORN nbivax nbi dk
  Date  Wed  8 Jul 1992 15 42  0200
 
         
json/hep-th-1992/9207020
 Paper  hep th 9205062
  From  sengupta iopb ernet in
  Date  Mon Ma

 Paper  hep th 9205008
  From  Jerome Gauntlett  jerome bohr uchicago edu 
  Date  Tue  5 May 92 16 
json/hep-th-1992/9205008
json/hep-th-1992/9202023
json/hep-th-1992/9204088
 Paper  hep th 9209051
  From  imsc suresh iitm ernet in  Suresh Govindarajan 
  Date  Tue  15 Sep 9
json/hep-th-1992/9209051
 Paper  hep th 9207057
  From  imsc partha iitm ernet in   Parthasarathi Majumdar 
  Date  Wed  15 J
json/hep-th-1992/9207057
json/hep-th-1992/9205042
 Paper  hep th 9208020
  From  makeenko itep msk su  Yuri Makeenko 
  Date  Thu  6 Aug 92 22 03 42  
json/hep-th-1992/9208020
 Paper  hep th 9207014
  From  YANG UORHEP bitnet cunyvm cuny edu
  Date  Sun  5 Jul 1992 14 35 EST

json/hep-th-1992/9207014
 Paper  9204033
  From  weiss physics ubc ca
  Date  13 Apr 92 20 57  0100
 
 
  documentstyle prepr
json/hep-th-1992/9204033
 Paper  9201063
  From  kamion guinness ias edu  Marc Kamionkowski 
  Date  Tue  28 Jan 92 17 05 04 
json/hep-th-1992/9201063
json/hep-th-1992/9205007
 Paper  9202003
  

json/hep-th-1992/9209028
 Paper  hep th 9208045
  From  MAKEENKO nbivax nbi dk
  Date  Mon  17 Aug 1992 15 42  0200
  Date  r
json/hep-th-1992/9208045
 Paper  hep th 9207074
  From   Stephen Naculich   naculich casa pha jhu edu 
  Date  22 Jul 92 15 3
json/hep-th-1992/9207074
json/hep-th-1992/9204071
 Paper  hep th 9205055
  From  Tetsuji Miwa  miwa kurims kyoto u ac jp 
  Date  Sat  16 May 92 15 57
json/hep-th-1992/9205055
 Paper  hep th 9205092
  From  gbarnich ulb ac be  Barnich Glenn 
  Date  Tue  26 May 92 12 11 25 ME
json/hep-th-1992/9205092
 Paper  hep th 9210012
  From  jhh waldzell physics yale edu  Jim Horne 
  Date  Thu  1 Oct 92 22 58
json/hep-th-1992/9210012
json/hep-th-1992/9207037
 Paper  hep th 9209029
  From  Andreas Honecker  UNP06B ibm rhrz uni bonn de 
  Date  Wed  09 Sep 92
json/hep-th-1992/9209029
 Paper  9204024
  From  Jal Panvel  JP1 LIVERPOOL AC UK vtvm2 cc vt edu 
  Date  Fri  10 Apr 92 15 4
json/hep-th-1992/9204024
 Paper  hep th 9211084
  From  SAGNOTTI rom

In [8]:
len(faulty)

4

In [9]:
re.sub(' +', ' ', "n$%^&*()ode!@#$%^&*(".translate(PUNCTUATION_TABLE)).strip()

'n ode'